# Imports & Notebook Setup

We will import the necessary packages utilized for modeling/evaluation in this notebook. We also make sure that our viualizations are exported to results/figures/modeling & evalutation/ directory, and metrics are exported to results/metrics directory:

In [1]:
import os
import sys 
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# Allow imports from ../src directory
sys.path.append(os.path.abspath(".."))
from src.train_models import (
    build_logistic_regression_model,
    build_random_forest_model,
    build_xgboost
    )
FIG_DIR = "../results/figures/modeling & evaluation/"
METRICS_DIR = "../results/metrics/"
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)
_plt_original_show = plt.show
_plt_fig_counter = {"count": 0}
def _save_and_show(*args, **kwargs) -> None:
    _plt_fig_counter["count"] += 1
    filename = os.path.join(FIG_DIR, f"figure_{_plt_fig_counter['count']:03d}.png")
    plt.savefig(filename, dpi=300, bbox_inches="tight")
    _plt_original_show(*args, **kwargs)
plt.show = _save_and_show

Compilation complete


- `pandas`: data wrangling and CSV loading
- `numpy`: array and numerical operations
- `matplotlib` & `seaborn`: visualizations and plots
- `os`, `sys`, `pathlib`: directory management and custom imports
- `sklearn`: train/test splitting and evaluation metrics. 
-  `src.train_models`: Logistic Regression, Random Forest and XGBoost.